In [1]:
import itertools, time

import pandas as pd
import numpy as np

In [2]:
class ConsoleChat:
    '''
    This is an interface for the chatbot.
    Turn off immersion mode in case you don't want to wait for all the bot typing.
    '''
    def __init__(self, bot, immersion_mode=True):
        self.bot = bot
        self.immersion_mode = immersion_mode
    def run(self):
        self.bot.reset()
                        
        while True:
            decision = self.bot.think()
            
            if decision == 'talk':
                if self.immersion_mode:
                    print('{}: '.format(self.bot.name), end='')
                    for character in self.bot.talk():
                        print(character, end='')
                        time.sleep(0.02)
                    print('')
                else:
                    print('{}: {}'.format(self.bot.name, self.bot.talk()))
                        
            elif decision == 'listen':
                self.bot.listen(input())
            elif decision == 'done':
                break
            else:
                raise ValueError('"{}" is not a viable bot decision.')

In [3]:
class ChatBot:
    '''
    Provide it with a name and a dialogue plan and it will talk.
    A dialogue plan should be a list of iterables which consist only of tuples of those two kinds:
    
    ('talk', a message that a bot will display)
    
    ('listen', a key in self.knowledge_dict where the user inputted message will be encoded.)
        
    Iterables in dialogue plan can be objects of user defined classes, but in case you want to add a reference
    to the chatbot object, please add a dialogue_plan after creating a chatbot is in example bellow.
    
    At the moment the only way to reach user messages is through self.knowledge_dict.
    
    '''
    def __init__(self, name, dialogue_plan=[]):
        self.name = name
        self.dialogue_plan = dialogue_plan
    
    def reset(self):
        self.knowledge_dict = {'self_name' : self.name}
        self.done = False
        self.dialogue_flow = itertools.chain.from_iterable(self.dialogue_plan)
        
    def think(self):
        try:
            self._action, self._text = next(self.dialogue_flow)
        except StopIteration:
            return 'done'
        return self._action
    
    def talk(self):
        return self._text.format(**self.knowledge_dict)
    
    def listen(self, user_message):
        self.knowledge_dict[self._text] = user_message

In [4]:
dialogue_start = [('talk', 'Hello! I\'m {self_name}. What\'s your name?'),
 ('listen', 'user_name'),
 ('talk', 'Hi {user_name}! How are you feeling today?'),
 ('listen', 'current_mood'),
 ('talk', 'Understood. Would you please answer a few questions?'),
 ('listen', 'agreement'),
 ('talk', 'Superb!')]

In [5]:
class BigFiveQuizzer:
    '''
    A very crude example of how to implement a question-answer style conversation.
    
    Asks n_questions random questions from csv table. After which it might comment on answers '1' or '5'
    based on relevant context.
    
    source: https://ipip.ori.org/new_ipip-50-item-scale.htm
    
    Goldberg, Lewis R. "The development of markers for the Big-Five factor structure."
    Psychological assessment 4.1 (1992): 26.
    
    '''
    def __init__(self, bot, n_questions = 5):
        self.big_five_questions = pd.read_csv('big_five.tsv', sep='\t')
        
        self.big_five_dimensions = ['extraversion',
                                    'agreeableness',
                                    'conscientiousness',
                                    'emotional stability',
                                    'intellect/imagination']
        self.bot = bot
        self.n_questions = n_questions
    def __iter__(self):
        yield 'talk', '''Please indicate for each statement whether it is
        1. Very Inaccurate,
        2. Moderately Inaccurate,
        3. Neither Accurate Nor Inaccurate,
        4. Moderately Accurate, or
        5. Very Accurate as a description of you.
    '''
        
        questions = self.big_five_questions.sample(self.n_questions)
        for index, row in questions.iterrows():
            while True:
                yield 'talk', row['statement']
                yield 'listen', 'big_five_last_answer'
                answer = self.bot.knowledge_dict['big_five_last_answer']
            
                if answer not in ['1','2','3','4','5']:
                    yield 'talk', 'Please only answer with a single number from 1 to 5.'
                else:
                    break
            
            if answer in '15':    
                
                dimension = self.big_five_dimensions[int(row['key'][0])-1]
            
                if (answer == '5') == (row['key'][1] == '+'):
                    value = 'describes you very well'
                else:
                    value = 'does not describe you at all'

                yield 'talk', 'Oh! It seems {} {}. Thank you for answering!'.format(dimension, value)
            else:
                yield 'talk', 'Got it! Thank you for answering.'

In [6]:
dialogue_end = [('talk', 'That would be all!'),
 ('talk', 'Thanks a lot for all the answers.'),
 ('talk', 'Would you be so kind and do one last thing for me?'),
 ('listen', 'agreement2'),
 ('talk', 'Tell me a single short story. Please!'),
 ('listen', 'story'),
 ('talk', 'Thanks! Was it a real story or a fictional one?'),
 ('listen', 'was_story_real'),
 ('talk', 'Thanks {user_name}, it was a pleasure talking to you! Let\'s do this again someday!')]

In [7]:
johnny = ChatBot('Johnny')

johnny.dialogue_plan = [dialogue_start, BigFiveQuizzer(johnny), dialogue_end]

In [8]:
cc = ConsoleChat(johnny)

In [9]:
cc.run()

Johnny: Hello! I'm Johnny. What's your name?
Marcin
Johnny: Hi Marcin! How are you feeling today?
Great!
Johnny: Understood. Would you please answer a few questions?
Sure
Johnny: Superb!
Johnny: Please indicate for each statement whether it is
        1. Very Inaccurate,
        2. Moderately Inaccurate,
        3. Neither Accurate Nor Inaccurate,
        4. Moderately Accurate, or
        5. Very Accurate as a description of you.
    
Johnny: Have frequent mood swings.
4
Johnny: Got it! Thank you for answering.
Johnny: Get chores done right away.
1
Johnny: Oh! It seems conscientiousness does not describe you at all. Thank you for answering!
Johnny: Don't talk a lot.
2
Johnny: Got it! Thank you for answering.
Johnny: Get upset easily.
3
Johnny: Got it! Thank you for answering.
Johnny: Seldom feel blue.
2
Johnny: Got it! Thank you for answering.
Johnny: That would be all!
Johnny: Thanks a lot for all the answers.
Johnny: Would you be so kind and do one last thing for me?
Yeah, y not :)
